# CSC 84040 - Project 3 - Graph Mining

## Project Description
In this project you will perform network analysis on a subset of an Amazon co-purchasing network created by [Leskovec et al. (2007)](http://www.cs.cmu.edu/~jure/pubs/viral-tweb.pdf).  The dataset contains various product networks including books, DVDs, and videos. It was collected by crawling the Amazon website in March 2003 according to *Customers Who Bought This Item Also Bought* product relationships. Thus, if product *A* is always co-purchased with product *B*, the graph contains a **directed** edge from *A* to *B*.

### Data
There are **two data files** associated with this project:
- *amazon_graph.txt*: This file contains the data for Part 1. It contains 16,552 observations (edges) of two (2) numbers representing product IDs. Each node represents a product and each directed edge between two nodes represents a co-purchase. The column fromNodeId contains the ID of the main purchasing item and ToNodeId contains the ID of the co-purchased items.
- *ratings_train.csv*: This file contains the *training* data for Part 2. It contains 8,250 observations of Amazon product ratings. The `avg_rating` column contains the average rating by Amazon customers on a scale of 0-5.

### Software Packages
The following Python software packages are required to complete the project:
- `pandas`
- `numpy`
- `scikit-learn`
- `matplotlib`
- `networkx`

## Part 1: Exploratory Network Analysis

Set the variable `GRAPH_FILE` to the **full path** to the Amazon product subgraph (**amazon_graph.txt**) on your system.

Part 1 of the project is designed to help you familiarize yourself with the dataset and basic concepts of network analysis. The insights gleaned from Part 1 will help in building the predictive models in Part 2.

### Network Analysis Tutorials
1. Read the article [Social Network Analysis in Python](https://www.datacamp.com/community/tutorials/social-network-analysis-python) to understand the basics of (social) network snalysis.
2. Review the [`NetworkX` Tutorial](https://networkx.org/documentation/stable/tutorial.html) to understand how you can use the `NetworkX` library to create and analyze graphs in Python.

### Network Analysis
After reading the above references, perform some basic network analyses and briefly explain each of your findings:
1. Load the **directed** network graph ($G$) from the `GRAPH_FILE`. You may use `NetworkX`'s [`read_edgelist` function](https://networkx.org/documentation/stable/reference/readwrite/generated/networkx.readwrite.edgelist.read_edgelist.html#read-edgelist) for this purpose.
   How many items are present in the network and how many co-purchases are present?
2. Compute the average shortest distance between all nodes in $G$. Explain your results briefly.
3. Compute the *transitivity* and the average *clustering coefficient* of the network graph $G$. Explain your findings briefly based on the definitions of clustering coefficient and transitivity.
4. Apply the PageRank algorithm to network $G$ with damping value 0.5 and find the 10 nodes with the highest PageRank. Explain your findings briefly.

## Additional Resources
1. [`NetworkX` API documentation](https://networkx.org/documentation/stable/reference/index.html)
2. You are welcome to review my article on [applying network analysis techniques to the mathematics collaboration graph](https://www.ams.org/journals/notices/202205/rnoti-p849.pdf).


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

#### Part 1

In [2]:
GRAPH_FILE = open("amazon_graph.txt", 'r')

1.

In [3]:
G = nx.read_edgelist(GRAPH_FILE)
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Number of items:",num_nodes)
print("Number of co-purchases:",num_edges)

Number of items: 11001
Number of co-purchases: 11025


2.

In [41]:
average_of = 0
count = 0

for C in (G.subgraph(c).copy() for c in nx.connected_components(G)):
    average_of += nx.average_shortest_path_length(C)
    count+=1

print("Average shortest distance:", average_of/count)

Average shortest distance: 1.243934564029527


The distance here shows that an item is usually co-purshased and sometimes if they aren't co-purchased, then one item is related to another by sharing the same item they are co-purchased with since the distance is more than one

3.

In [42]:
print("Transitivity:",nx.transitivity(G))

Transitivity: 0.5251489080079418


The transtivity show that of the triads form (one item being co-purchased with one of two items), 0.525 of them have a chance that if an item is co-purchase with one of 2 items, then those two items are also likely to be co-purchased with each other

In [43]:
print("Average clustering coefficient:",nx.average_clustering(G))

Average clustering coefficient: 0.4000347163833983


The average clustering coefficient indicate that given a subgraph with an item (co-purchases with that item), the average degree (coefficient to indicate completness of a graph) of all items in the subgraph being connected to each other is 0.4. 0.4 of the nodes in their subgraph won't make the subgraph complete. 

4.

In [44]:
pr = nx.pagerank(G, alpha=0.5)
pr_sorted = dict(sorted(pr.items(), key=lambda item: item[1], reverse=True))

#obtain top 10 items
sorted_10 = list(pr_sorted.items())[:10]

for element in sorted_10:
    print(element)

('4429', 0.00036442443488212214)
('33', 0.0003598057652970889)
('2501', 0.0003466387704726638)
('7303', 0.00032008729015646807)
('9106', 0.00028172941894020575)
('3464', 0.00023908362410481076)
('151', 0.0002334026017886055)
('10745', 0.00021860125441267042)
('16340', 0.0002110588786436082)
('18', 0.00020506390788164124)


An item purchased will most likely direct a buyer to look at those 10 items. Item 4429 is most likely to get seen after looking at an item.

## Part 2: Predict Product Ratings using Network-derived Features

Set the variable `RATINGS_TRAIN_FILE` to the **full path** to the Amazon product ratings file *ratings_train.csv*.

In this part of the project, you will build a machine learning model to predict the average rating of Amazon products on a scale of 0-5 using various network properties as features.

The training dataset *ratings_train.csv* contains the product IDs and average ratings, the latter of which will be the targets ($y$ values) of your machine learning models.

For each product ID in the training set, you need to extract at least **four (4)** different features based on its network properties to train your model. Some of the features that you can consider using include:
- Clustering Coefficient
- Page Rank
- Degree centrality
- Closeness centrality
- Betweenness centrality
- Degree of the node

Using the extracted features, you will build at least **two (2)** regression models, where the input values ($x$ values) will be the extracted features for each product, and the regression targets ($y$ values) will be the average rating for said product. Some of the *regression* models that you can consider using include:
- [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- Support Vector Machine (SVM)
- Random forest
- Multi-layer perceptron

The main deliverable for Part 2 is a step-by-step analysis of your feature extraction, selection, and model building exercise, describing clearly how you generated features from your dataset and why you chose specific features over the others. You should evaluate the performance of multiple combinations of models and feature sets, using [mean absolute error (MAE)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) as the performance metric.

Your Jupyter notebook should contain the reproducible code for training various models as well as text descriptions of your conclusions after each step.

Your grade on this part of the homework will depend on the accuracy of your model on a holdout test set (**not provided**) as well as your step-by-step description of how you arrived at your final model. We will evaluate your model using MAE on the test set.

## Supervised Learnign in Scikit-learn
Given a training dataset $(\mathcal{X},\mathcal{Y})$, the goal of **supervised learning** is to learn a function $f$ that maps inputs to outputs, i.e., $f:\mathbf{x}\rightarrow y$. For this problem, the $\mathbf{x}\in\mathcal{X}$ values will be vectors of 4 or more features derived from the network, while the $y\in \mathcal{Y}$ values will be the average rating. The general process for training a model in `Scikit-learn` is as follows (using `LinearRegression` as an example):

1. `lr = LinearRegression(fit_intercept=True) #Instantiate an object from the model class, setting any parameters as desired.`
2. `lr.fit(X_train,Y_train) #Train the model using the training data.`
3. `Y_train_pred = lr.predict(X_train) #Apply the model to a dataset, generating labels.`
4. `mae_train = mean_absolute_error(Y_train,Y_train_pred) #Test model performance based on an appropriate metric.`

Line 4 above measures the performance of the model on the same data that it was trained with. This is to gauge how well the model fit the data and provides a baseline for performance when you apply the model to the test set. Typically, model performance new, unseen data (e.g., the test set) will decrease. This decrease is the *generalization error* of the model.

## Additional Resources
- [Scikit-learn API](https://scikit-learn.org/stable/modules/classes.html)
- [Scikit-learn User Guide: Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
- [Scikit-learn User Guide: Linear Models for Regression](https://scikit-learn.org/stable/modules/linear_model.html)

#### Part 2

In [5]:
RATINGS_TRAIN_FILE = pd.read_csv("ratings_train.csv")

#### Step 1: Create features to predict the average rating: 

Feature to be created are:
- The degree of centrality of each item
- The degree of clossness for each item
- The degree of betweeness for each item
- The degree of an item
- The rank of an item
- The cluster coefficient of each item


These features were created based on properties of nodes in a graph. The use of networkx returns the id of each item along with the property of the item.

In [6]:
degree_cen = nx.degree_centrality(G)
closeness_cen = nx.closeness_centrality(G)
between_cen = nx.betweenness_centrality(G)
degree = nx.degree(G)
pr = nx.pagerank(G, alpha=0.5)

In [7]:
cluster_num = nx.clustering(G)

#### Step 1.5: Create a dataframe with the train file and the created features

- Create a dataframe of each of the created features.
- This will then be merged with the training set where the items are the same for all of the created features.
- Want to convert the id into a int to avoid any error with merging.

In [8]:
# Convert centrality measures and clustering coefficient to DataFrames
degree_cen_df = pd.DataFrame(list(degree_cen.items()), columns=['id', 'degree_centrality'])
closeness_cen_df = pd.DataFrame(list(closeness_cen.items()), columns=['id', 'closeness_centrality'])
between_cen_df = pd.DataFrame(list(between_cen.items()), columns=['id', 'betweenness_centrality'])
pr_df = pd.DataFrame(list(pr.items()), columns=['id', 'pagerank'])
cluster_num_df = pd.DataFrame(list(cluster_num.items()), columns=['id', 'clustering_coefficient'])
degree_df = pd.DataFrame(list(dict(degree).items()), columns=['id', 'degree'])

In [9]:
# Convert centrality measures to DataFrames
degree_cen_df['id'] = degree_cen_df['id'].astype('int64')
closeness_cen_df['id'] = closeness_cen_df['id'].astype('int64')
between_cen_df['id'] = between_cen_df['id'].astype('int64')
pr_df['id'] = pr_df['id'].astype('int64')
cluster_num_df['id'] = cluster_num_df['id'].astype('int64')

# Convert degree to a DataFrame
degree_df['id'] = degree_df['id'].astype('int64')

# Merge all the DataFrames
merged_df = RATINGS_TRAIN_FILE.merge(degree_cen_df, on='id').merge(closeness_cen_df, on='id').merge(between_cen_df, on='id').merge(pr_df, on='id').merge(cluster_num_df, on='id').merge(degree_df, on='id')

In [10]:
merged_df

,id,group,avg_rating,degree_centrality,closeness_centrality,betweenness_centrality,pagerank,clustering_coefficient,degree
0,93183,Book,4.5,0.000091,0.000264,0.000000e+00,0.000064,0.000000,1
1,231872,Book,4.0,0.000182,0.000182,0.000000e+00,0.000091,1.000000,2
2,67584,Book,4.0,0.000182,0.000182,1.653043e-08,0.000119,0.000000,2
3,116605,Book,4.0,0.000182,0.000182,0.000000e+00,0.000091,1.000000,2
4,33688,Book,4.0,0.000182,0.000182,0.000000e+00,0.000091,1.000000,2
...,...,...,...,...,...,...,...,...,...
8245,118615,Video,3.5,0.000182,0.000182,0.000000e+00,0.000091,1.000000,2
8246,10235,Book,5.0,0.000273,0.000291,6.612171e-08,0.000113,0.333333,3
8247,102410,Book,4.0,0.000091,0.000091,0.000000e+00,0.000091,0.000000,1
8248,161184,Book,5.0,0.000182,0.000205,0.000000e+00,0.000088,1.000000,2


#### Step 2: Select features for modeling

The features were selected based on the linear relationship is between with the avg_rating and with the other features.
Features that didn't have a string relationship (close to 0 meant that the feature didn't have any relation with the data). The features chosen were degee centrality, betweeness centrality, page rank and clustering coefficient. Degree was excluded because of how similiar it is to degree centrality and closeness centrality was excluded becuase of how close it is to 0. 

In [11]:
# correlation matrix
final_df = merged_df.drop(columns=['id','group']) #get rid of uneeded columns
corr_matrix = final_df.corr()
corr_matrix

,avg_rating,degree_centrality,closeness_centrality,betweenness_centrality,pagerank,clustering_coefficient,degree
avg_rating,1.000000,0.015388,-0.000183,-0.012372,0.007071,0.008893,0.015388
degree_centrality,0.015388,1.000000,0.277528,0.386463,0.670863,0.380754,1.000000
closeness_centrality,-0.000183,0.277528,1.000000,0.325786,0.036011,-0.053539,0.277528
betweenness_centrality,-0.012372,0.386463,0.325786,1.000000,0.305633,-0.062187,0.386463
pagerank,0.007071,0.670863,0.036011,0.305633,1.000000,-0.048652,0.670863
clustering_coefficient,0.008893,0.380754,-0.053539,-0.062187,-0.048652,1.000000,0.380754
degree,0.015388,1.000000,0.277528,0.386463,0.670863,0.380754,1.000000


#### Step 3: Experimenting data with different models

Different regression models were used to test their performance with the dataset.

The list of models experimented were:
- Linear regression
- Random forest regression
- stocchastic gradient descent
- Support vector machine (linear and rbf kernal)
- multi-layer perception

In [12]:
from sklearn.model_selection import train_test_split

# Partition data
X = merged_df[['degree_centrality', 'betweenness_centrality', 'pagerank', 'clustering_coefficient']]
y = merged_df['avg_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

For the use of a test set, test_size is the percentage of the test set with the merged data.

In [19]:
X_train_np = X_train.values
y_train_np = y_train.values

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_absolute_error

In [21]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def k_fold_cross_validation(model, X, y, n_folds=5, random_state=50):
    
    # Initialize k-fold cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    
    # Initialize a list to store MAE for each fold
    mae_scores = []
    
    # Loop over each fold
    for train_index, test_index in kf.split(X):
        # Split the data into training and test sets for this fold
        X_train_fold, X_val_fold = X[train_index], X[test_index]
        y_train_fold, y_val_fold = y[train_index], y[test_index]
        
        # Initialize the model
        model_instance = model
        
        # Fit the model on the training data for this fold
        model_instance.fit(X_train_fold, y_train_fold)
        
        # Predict on the validation set for this fold
        y_pred_fold = model_instance.predict(X_val_fold)
        
        # Calculate the MAE for this fold and append it to the list of scores
        mae = mean_absolute_error(y_val_fold, y_pred_fold)
        mae_scores.append(mae)
    
    # Calculate the average MAE across all folds
    average_mae = np.mean(mae_scores)
    
    return average_mae

In [22]:
#Linear regression
model_slm  = LinearRegression()
average_mae_slm = k_fold_cross_validation(model_slm, X_train_np, y_train_np)
print("Average MAE:", average_mae_slm)

Average MAE: 0.5288786133532613


In [23]:
#random forest
model_rf = RandomForestRegressor()
average_mae_rf = k_fold_cross_validation(model_rf, X_train_np, y_train_np)
print("Average MAE:", average_mae_rf)

Average MAE: 0.5431610507943818


In [24]:
#SVM Linear kernal
model_svr = SVR(kernel = 'linear')
average_mae_svr = k_fold_cross_validation(model_svr, X_train_np, y_train_np)
print("Average MAE:", average_mae_svr)

Average MAE: 0.5065604516667258


In [31]:
#SVM rbf kernal
model_svr2 = SVR(kernel = 'rbf')
average_mae_svr = k_fold_cross_validation(model_svr2, X_train_np, y_train_np)
print("Average MAE:", average_mae_svr)

Average MAE: 0.50654556756931


In [37]:
from sklearn.linear_model import SGDRegressor

# Stochastic gradient descent regressor
sgd_regressor = SGDRegressor(learning_rate='constant', max_iter=8000, tol=1e-3, random_state=50)
average_mae_sgd = k_fold_cross_validation(sgd_regressor, X_train_np, y_train_np)
print("Average MAE:", average_mae_sgd)

Average MAE: 0.5289805294331418


In [39]:
# Gradient boosting
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

average_mae_gbr = k_fold_cross_validation(gbr, X_train_np, y_train_np)
print("Average MAE:", average_mae_gbr)

Average MAE: 0.5315212579149267


In [26]:
from sklearn.neural_network import MLPRegressor

# MLP regressor
mlp_regressor = MLPRegressor(hidden_layer_sizes=(100,50), activation='relu', solver='adam', max_iter=500, random_state=50)
average_mae_mlp = k_fold_cross_validation(mlp_regressor, X_train_np, y_train_np)
print("Average MAE:", average_mae_mlp)

Average MAE: 0.5295793169366823


#### Step 4: Model Selection

In [15]:
#SVM with rbf kernal
model_svr2 = SVR(kernel = 'rbf')
model_svr2.fit(X, y)
y_pred = model_svr2.predict(X)
print("SVR MAE:", mean_absolute_error(y, y_pred))

SVR MAE: 0.5107267897779526


For my model selection, I chose the support vector machine with a rbf kernal because of how low the average MAE was during cross validation; between the linear kernal and rbf kernal, the rbf was chosen since it can predict values based on non-linear boundaries. When applying the model, the MAE was also lower than most of the average MAE during cross validation. 

**In case a test set is used, changed the variables in case the data is partitioned**

In [ ]:
#SVM with rbf kernal for partitioned data
model_svr2 = SVR(kernel = 'rbf')
model_svr2.fit(X_train, y_train)
y_pred = model_svr2.predict(X_test)
print("SVR MAE:", mean_absolute_error(y_test, y_pred))